**Run the following 2 cells to install `fastell4py` and restart kernel to continue from 3rd cell**

In [ ]:
!git clone https://github.com/sibirrer/fastell4py.git
%cd fastell4py
!python setup.py install --user
%cd ..

Cloning into 'fastell4py'...
remote: Enumerating objects: 75, done.
remote: Total 75 (delta 0), reused 0 (delta 0), pack-reused 75
Unpacking objects: 100% (75/75), done.
/content/drive/My Drive/fastell4py
running install
running bdist_egg
running egg_info
running build_src
build_src
building extension "fastell4py._fastell" sources
creating build
creating build/src.linux-x86_64-3.7
f2py options: []
f2py: fastell.pyf
Reading fortran codes...
	Reading file 'fastell.pyf' (format:free)
Post-processing...
	Block: _fastell
			Block: ellipphi
			Block: ellipphi_array
			Block: fastellmag
			Block: fastellmag_array
			Block: fastelldefl
			Block: fastelldefl_array
Post-processing (stage 2)...
Building modules...
	Building module "_fastell"...
		Constructing wrapper function "ellipphi"...
		  ellipphi(x1,x2,q,gam,arat,s2,phi)
		Constructing wrapper function "ellipphi_array"...
		  ellipphi_array(x1,x2,q,gam,arat,s2,phi,[n])
		Constructing wrapper function "fastellmag"...
		  fastellmag(x1in,x2in

In [ ]:
!pip install lenstronomy==1.0.0  # tested with version 1.9.0
!pip install corner  schwimmbad

     |████████████████████████████████| 5.8 MB 22.6 MB/s 
  Created wheel for lenstronomy: filename=lenstronomy-1.0.0-py3-none-any.whl size=5963869 sha256=6cf47c3bff1c0e8839a213f38268d749ada7d09641404b2e5feb9ab18c87d39d
  Stored in directory: /root/.cache/pip/wheels/04/bd/82/6559cebd4fc5e48f79c624969674ec640da1ad25e351299b8c
Successfully built lenstronomy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for schwimmbad: filename=schwimmbad-0.3.2-py3-none-any.whl size=15744 sha256=8a841b6dd1172373d1d6d9e362fb47988964679a632c081a040ac34cd6f07587
  Stored in directory: /root/.cache/pip/wheels/37/63/1c/16eaacde21747bfd9f8862220535a0ab502c35029536f306c3
Successfully built schwimmbad


**Resume from here**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

ValueError: ignored

In [ ]:
# import of standard python libraries
import numpy as np
import pandas as pd
import os
import time
import corner
from astropy.io import fits
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm


# import lenstronomy package
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.ImSim.image_model import ImageModel
import lenstronomy.Util.param_util as param_util
import lenstronomy.Util.simulation_util as sim_util
import lenstronomy.Util.image_util as image_util
from lenstronomy.Util import kernel_util
from lenstronomy.Data.imaging_data import ImageData
from lenstronomy.Data.psf import PSF

%matplotlib inline

In [ ]:
num_sample = 90000 # 
show_img = False
dest_dir = '/content/drive/MyDrive/Lensing_Sim_Data/dev_1w' #'s3://joshualin24/dev_256_120000/' #


img_config = {
    'exp_time': 1000.,  #  exposure time (arbitrary units, flux per pixel is in units #photons/exp_time unit)
    'numPix': 300,  #  cutout pixel size per axis
    'deltaPix': 0.02,  #  pixel size in arcsec (area per pixel = deltaPix**2)
    'fwhm': 0.07  # full width at half maximum of PSF (Chen et al. 2019, 2020)
     #'sigma_bkg': np.random.uniform(0.04, 4),  #  background noise per pixel (Gaussian)
}

In [ ]:
def get_lens_bnn_config(params):
    config = {}
    config['gamma_ext'] = np.maximum(np.random.normal(params['gamma_ext_mu'], params['gamma_ext_sigma']), 0)
    config['psi_ext'] = np.random.uniform(0.0, 2.*np.pi)
    config['theta_E'] = np.maximum(np.random.normal(loc=params['theta_E_mu'], scale=params['theta_E_sigma']), 0.1)
    config['gamma'] = np.maximum(np.random.normal(params['gamma_mu'], params['gamma_sigma']), 1.85)
    config['center_x'] = np.random.normal(params['center_mu'], params['center_sigma'])
    config['center_y'] = np.random.normal(params['center_mu'], params['center_sigma'])
    config['e1'] = np.minimum(np.random.normal(params['e_mu'], params['e_sigma']), 0.9)
    config['e2'] = np.minimum(np.random.normal(params['e_mu'], params['e_sigma']), 0.9)
    config['model_list'] = params['model_list']
    return config


def create_lens_model(config):
    kwargs_shear = {
        'gamma_ext': config['gamma_ext'], 
        'psi_ext': config['psi_ext'],
    }  # shear values to the source plane
    kwargs_spemd = {
        'theta_E': config['theta_E'], 
        'gamma': config['gamma'], 
        'center_x': config['center_x'], 
        'center_y': config['center_y'], 
        'e1': config['e1'], 
        'e2': config['e2'],
    }  # parameters of the deflector lens model

    kwargs_maps = {
        'SPEMD': kwargs_spemd,
        'SHEAR_GAMMA_PSI': kwargs_shear,
    }

    # the lens model is a supperposition of an elliptical lens model with external shear
    res = {
        'kwargs_list': [kwargs_maps[key] for key in kwargs_maps],
        'class': LensModel(lens_model_list=config['model_list']),
    }
    return res



def create_light_model(config):
    #e1, e2 = param_util.phi_q2_ellipticity(config['phi_G'], config['q'])
    R_sersic = np.random.normal(config['R_sersic_mu'], config['R_sersic_sigma'])
    n_sersic = np.random.normal(config['n_sersic_mu'], config['n_sersic_sigma'])

    kwargs = {
        'amp': config['amp'],
        'R_sersic': R_sersic,
        'n_sersic': n_sersic,
        'e1': config['e1'],
        'e2': config['e2'],
        'center_x': config['center_x'],
        'center_y': config['center_y'],
    }

    res = {
        'kwargs_list': [kwargs],
        'class': LightModel(light_model_list=config['model_list']),
    }
    return res


def get_source_position(params):
    res = {
        'x': np.random.normal(params['position_mu'], params['position_sigma']),
        'y': np.random.normal(params['position_mu'], params['position_sigma']),
    }
    return res


def create_point_source_model(lens_model, source_position, img_config, quasar_amp):
    lensEquationSolver = LensEquationSolver(lens_model['class'])
    x_image, y_image = lensEquationSolver.findBrightImage(
        source_position['x'], 
        source_position['y'], 
        lens_model['kwargs_list'], 
        numImages = 4,
        min_distance = img_config['deltaPix'], 
        search_window = img_config['numPix'] * img_config['deltaPix'],
    )
    
    mag = lens_model['class'].magnification(x_image, y_image, kwargs=lens_model['kwargs_list'])

    # quasar point source position in the source plane and intrinsic brightness
    kwargs = {
        'ra_image': x_image, 
        'dec_image': y_image,
        'point_amp': np.abs(mag) * quasar_amp,
    }  
    point_source_class = PointSource(
        point_source_type_list=['LENSED_POSITION'], 
        fixed_magnification_list=[False],
    )

    res = {
        'kwargs_list': [kwargs],
        'class': point_source_class,
    }
    return res, len(x_image)


def plot_image(img):
    cmap_string = 'gray'
    cmap = plt.get_cmap(cmap_string)
    cmap.set_bad(color='k', alpha=1.)
    cmap.set_under('k')

    v_min = -1
    v_max = 2

    f, axes = plt.subplots(1, 1, figsize=(6, 6), sharex=False, sharey=False)
    ax = axes
    im = ax.matshow(np.log10(img), origin='lower', vmin=v_min, vmax=v_max, cmap=cmap, extent=[0, 1, 0, 1])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.autoscale(False)
    plt.show()

In [ ]:
meta_keys = ['img_path', 
    'theta_E', 'gamma', 'center_x', 'center_y', 'e1', 'e2', 'gamma_ext', 'psi_ext', 
    'source_x', 'source_y', 'source_n_sersic', 'source_R_sersic', 'sersic_source_e1', 'sersic_source_e2', 
    'lens_light_n_sersic', 'lens_light_R_sersic', 'num_quasars', 'sigma_bkg']
meta_df = {key: np.zeros(num_sample) for key in meta_keys}
meta_df['img_path'] = ['placeholder'] * num_sample

In [ ]:
def diff_PSF(num_sample, i):
    mark = num_sample/6
    if i < 3*mark:
      num_pixel = 300
      delta_pixel = 0.02
      quasar_amp = 1500
    else:
      num_pixel = 120
      delta_pixel = 0.05
      quasar_amp = 1500
    if i < mark:
      Real_PSF=fits.getdata('/content/drive/MyDrive/PSF/J0924_AO_PSF_20mas.fits')
    elif i >= mark and i < 2*mark:
      Real_PSF=fits.getdata('/content/drive/MyDrive/PSF/HE0435_AO_PSF_20mas.fits')
    elif i >= 2*mark and i < 3*mark:
      Real_PSF=fits.getdata('/content/drive/MyDrive/PSF/PG1115_AO_PSF_20mas.fits')
    elif i >= 3*mark and i < 4*mark:
      Real_PSF=fits.getdata('/content/drive/MyDrive/PSF/PSF_-2_6.fitsnew.fits')
    elif i >= 4*mark and i < 5*mark:
      Real_PSF=fits.getdata('/content/drive/MyDrive/PSF/PSF_-1_4.fitsnew.fits')
    elif i >= 5*mark and i < 6*mark:
      Real_PSF=fits.getdata('/content/drive/MyDrive/PSF/PSF_-0.5_0.fitsnew.fits')
    return num_pixel, delta_pixel, quasar_amp, Real_PSF

In [ ]:
for i in tqdm(range(num_sample)):
    sigma_bkg = np.random.uniform(0.04, 4)
    img_config['numPix'], img_config['deltaPix'], quasar_amp, real_PSF = diff_PSF(num_sample, i)

    # Generate the coordinate grid and image properties kwargs
    kwargs_data = sim_util.data_configure_simple(
        img_config['numPix'], 
        img_config['deltaPix'], 
        img_config['exp_time'], 
        sigma_bkg,
    )
    image_data_class = ImageData(**kwargs_data)

    #kwargs_psf = {
    #    'psf_type': 'GAUSSIAN', 
    #    'fwhm': img_config['fwhm'], 
    #    'pixel_size': img_config['deltaPix'], 
    #    'truncation': 3,
    #}
    kwargs_psf = {
        'psf_type': 'PIXEL', 
        'kernel_point_source':real_PSF
    }
    psf_class = PSF(**kwargs_psf)


    # Mean of the BNN prior: mass distribution for the lens system (all kinds of mass: dm halo + galaxy )
    lens_bnn_config_params = {
        'gamma_ext_mu': 0.015, 
        'gamma_ext_sigma': 0.005, 
        'theta_E_mu': 1.25, 
        'theta_E_sigma': 0.3, 
        'gamma_mu': 2.0, 
        'gamma_sigma': 0.2, 
        'center_mu': 0.0, 
        'center_sigma': 0.02, 
        'e_mu': 0.00,
        'e_sigma': 0.2,
        'model_list': ['SPEMD', 'SHEAR_GAMMA_PSI'],

    }
    lens_bnn_config = get_lens_bnn_config(lens_bnn_config_params)
    lens_model = create_lens_model(lens_bnn_config)


    # lens light model
    lens_light_config = {
        'e1': lens_bnn_config['e1'], # taken from lens
        'e2': lens_bnn_config['e2'], # taken from lens
        'R_sersic_mu': 1.0,
        'R_sersic_sigma': 0.05,
        'n_sersic_mu': 4.0,
        'n_sersic_sigma': 0.1,
        'amp': 1000.0,
        'center_x': lens_bnn_config['center_x'],  # taken from lens
        'center_y': lens_bnn_config['center_y'],  # taken from lens
        'model_list': ['SERSIC_ELLIPSE'],
    }
    lens_light_model = create_light_model(lens_light_config)


    source_position_params = {
        'position_mu': 0.0,
        'position_sigma': 0.1,
    }
    source_position = get_source_position(source_position_params)



    # source light model
    source_sersic_config = {
        'e1': 0.,
        'e2': 0.5,
        'R_sersic_mu': 0.2,
        'R_sersic_sigma': 0.01,
        'n_sersic_mu': 0.5,
        'n_sersic_sigma': 0.05,
        'amp': 5000.2,
        'center_x': source_position['x'],
        'center_y': source_position['y'],
        'model_list': ['SERSIC_ELLIPSE'],
    }
    source_model = create_light_model(source_sersic_config)


    # quasar point source model
    point_source_model, num_quasars = create_point_source_model(lens_model, source_position, img_config, quasar_amp)

    # not sure what this is for
    # (Chih-Fan: this is for subsampling the PSF since the pixel size of the 
    #  HST detector is too fat to properly sample the PSF FHWM)
    kwargs_numerics = {'supersampling_factor': 1}


    imageModel = ImageModel(
        image_data_class, 
        psf_class, 
        lens_model['class'], 
        source_model['class'],
        lens_light_model['class'], 
        point_source_model['class'], 
        kwargs_numerics = kwargs_numerics,
    )


    # generate image
    img = imageModel.image(
        lens_model['kwargs_list'], 
        source_model['kwargs_list'], 
        lens_light_model['kwargs_list'], 
        point_source_model['kwargs_list'],
    )
    poisson = image_util.add_poisson(img, exp_time=img_config['exp_time'])
    bkg = image_util.add_background(img, sigma_bkd=sigma_bkg)
    img = img  + bkg + poisson

    image_data_class.update_data(img)
    kwargs_data['image_data'] = img


    # visualize images
    if show_img == True:
        plot_image(img)
    #print(num_quasars)


    # save image file
    img_path = os.path.join(dest_dir, 'X_{0:07d}.npy'.format(i))  # changed to 0-index
    np.save(img_path, img)
    #!aws s3 cp img_path s3://joshualin24/dev_256_120000/


    # fill in meta_df
    meta_df['img_path'][i] = img_path
    
    # lens keys
    _lens_dict = {
        **lens_model['kwargs_list'][0],
        **lens_model['kwargs_list'][1],
    }
    for key in ['theta_E', 'gamma', 'center_x', 'center_y', 'e1', 'e2', 'gamma_ext', 'psi_ext']:
        meta_df[key][i] = _lens_dict[key]
    
    # source keys
    _source_dict = source_model['kwargs_list'][0]
    meta_df['source_x'][i] = _source_dict['center_x']
    meta_df['source_y'][i] = _source_dict['center_y']
    meta_df['source_n_sersic'][i] = _source_dict['n_sersic']
    meta_df['source_R_sersic'][i] = _source_dict['R_sersic']
    meta_df['sersic_source_e1'][i] = _source_dict['e1']
    meta_df['sersic_source_e2'][i] = _source_dict['e2']

    # lens light keys
    _lens_lights_dict = lens_light_model['kwargs_list'][0]
    #meta_df['lens_light_e1'][i] = _lens_lights_dict['e1']
    #meta_df['lens_light_e2'][i] = _lens_lights_dict['e2']
    meta_df['lens_light_n_sersic'][i] = _lens_lights_dict['n_sersic']
    meta_df['lens_light_R_sersic'][i] = _lens_lights_dict['R_sersic']

    # number of quasar
    meta_df['num_quasars'][i] = int(num_quasars)

    # background noise
    meta_df['sigma_bkg'][i] = sigma_bkg

# save meta data df as csv
meta_df = pd.DataFrame.from_dict(meta_df)
meta_path = os.path.join(dest_dir, 'metadata.csv')
meta_df.to_csv(meta_path, index=None)
#!aws s3 cp meta_path s3://joshualin24/dev_256_120000/

print(meta_df.columns.values)
print(meta_df)

100%|██████████| 90000/90000 [5:42:11<00:00,  4.38it/s]


['img_path' 'theta_E' 'gamma' 'center_x' 'center_y' 'e1' 'e2' 'gamma_ext'
 'psi_ext' 'source_x' 'source_y' 'source_n_sersic' 'source_R_sersic'
 'sersic_source_e1' 'sersic_source_e2' 'lens_light_n_sersic'
 'lens_light_R_sersic' 'num_quasars' 'sigma_bkg']
                                                img_path   theta_E     gamma  \
0      /content/drive/MyDrive/Lensing_Sim_Data/dev_1w...  1.250506  2.113448   
1      /content/drive/MyDrive/Lensing_Sim_Data/dev_1w...  1.611301  1.850000   
2      /content/drive/MyDrive/Lensing_Sim_Data/dev_1w...  1.335334  2.032483   
3      /content/drive/MyDrive/Lensing_Sim_Data/dev_1w...  1.678901  2.057198   
4      /content/drive/MyDrive/Lensing_Sim_Data/dev_1w...  1.292377  1.850000   
...                                                  ...       ...       ...   
89995  /content/drive/MyDrive/Lensing_Sim_Data/dev_1w...  0.856817  2.055607   
89996  /content/drive/MyDrive/Lensing_Sim_Data/dev_1w...  0.991076  1.850000   
89997  /content/drive/MyDr